In [1]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteFile, CogniteFileApply
from cognite.client.exceptions import CogniteException
from IPython.display import display

from tests.auth_utils import get_toolkit_client

In [2]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [3]:
SPACE = "sp_upgrade"

In [4]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Mismatch Cognite/Asset-Centric

In [5]:
file = FileMetadataWrite(
    external_id="ts_case3_mismatch2",
    name="Case 3 Mismatch Cognite-Asset-Centric",
)

In [6]:
# Ensure the file does not exist before running the test
try:
    client.files.delete(external_id=file.external_id, ignore_unknown_ids=True)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
    try:
        client.data_modeling.instances.delete(CogniteFileApply(space=SPACE, external_id=file.external_id).as_id())
    except CogniteException as e:
        print(f"\033[91mError\033[0m {e.code} {e.message!s}")
    else:
        print("\033[92mFile node deleted successfully\033[0m")
else:
    print(f"\033[92mFile with external_id {file.external_id} deleted successfully or did not exists\033[0m")

Error 403 Resource not found. This may also be due to insufficient access rights.
File node deleted successfully


In [7]:
created, _ = client.files.create(file)
created

,value
external_id,ts_case3_mismatch2
name,Case 3 Mismatch Cognite-Asset-Centric
labels,[]
id,8959548951294367
created_time,2025-07-18 13:03:13.514000
last_updated_time,2025-07-18 13:03:13.514000
uploaded,False


In [8]:
filecontent = b"This is filecontent"

In [9]:
client.files.upload_content_bytes(filecontent, external_id=created.external_id)

,value
external_id,ts_case3_mismatch2
name,Case 3 Mismatch Cognite-Asset-Centric
id,8959548951294367
created_time,2025-07-18 13:03:13.514000
last_updated_time,2025-07-18 13:03:13.514000
uploaded,False


In [10]:
try:
    updated = client.files.set_pending_ids((SPACE, file.external_id), external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    display(updated)

,value
external_id,ts_case3_mismatch2
name,Case 3 Mismatch Cognite-Asset-Centric
id,8959548951294367
created_time,2025-07-18 13:03:13.514000
last_updated_time,2025-07-18 13:03:16.225000
uploaded,True
uploaded_time,2025-07-18 13:03:15.130000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."


In [11]:
cognite_file = CogniteFileApply(
    space=SPACE,
    external_id=file.external_id,
    name="Another name for Case 3 Mismatch",
)

In [12]:
created_dm = client.data_modeling.instances.apply(cognite_file).nodes
created_dm

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,sp_upgrade,node,ts_case3_mismatch2,1,True,2025-07-18 13:03:18.564,2025-07-18 13:03:18.564


In [13]:
retrieved_content = client.files.download_bytes(instance_id=created_dm[0].as_id())

In [14]:
retrieved_content

b'This is filecontent'

In [15]:
retrieved_file = client.files.retrieve(external_id=file.external_id)

In [16]:
retrieved_file

,value
external_id,ts_case3_mismatch2
instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."
name,Case 3 Mismatch Cognite-Asset-Centric
id,8959548951294367
created_time,2025-07-18 13:03:13.514000
last_updated_time,2025-07-18 13:03:16.225000
uploaded,True
uploaded_time,2025-07-18 13:03:15.130000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."


In [17]:
retrieved_cognite_file = client.data_modeling.instances.retrieve_nodes(cognite_file.as_id(), node_cls=CogniteFile)

In [18]:
retrieved_cognite_file.dump()

{'space': 'sp_upgrade',
 'externalId': 'ts_case3_mismatch2',
 'version': 1,
 'lastUpdatedTime': 1752843798564,
 'createdTime': 1752843798564,
 'instanceType': 'node',
 'properties': {'cdf_cdm': {'CogniteFile/v1': {'name': 'Another name for Case 3 Mismatch',
    'description': None,
    'tags': None,
    'aliases': None,
    'sourceId': None,
    'sourceContext': None,
    'source': None,
    'sourceCreatedTime': None,
    'sourceUpdatedTime': None,
    'sourceCreatedUser': None,
    'sourceUpdatedUser': None,
    'assets': None,
    'mimeType': None,
    'directory': None,
    'isUploaded': False,
    'uploadedTime': None,
    'category': None}}}}

In [19]:
retrieved_cognite_ts_classic = client.files.retrieve(instance_id=cognite_file.as_id())
retrieved_cognite_ts_classic

,value
external_id,ts_case3_mismatch2
instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."
name,Case 3 Mismatch Cognite-Asset-Centric
id,8959548951294367
created_time,2025-07-18 13:03:13.514000
last_updated_time,2025-07-18 13:03:16.225000
uploaded,True
uploaded_time,2025-07-18 13:03:15.130000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."
